In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers
!pip install accelerate==0.29.0
!pip install datasets==2.15.0
!pip install peft==0.7.1
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successf

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
# from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np

In [3]:
df_explanations = pd.read_csv('/kaggle/input/dataset/Dataset/explanations.csv')

def remove_jpg_extension(image_name):
    final_image_name = image_name.split('.')[0]
    return final_image_name

df_explanations['image'] = df_explanations['image'].apply(remove_jpg_extension)

df_explanations.drop('Unnamed: 2',axis=1,inplace=True)

In [4]:
df_explanations.columns

Index(['image', 'llm_explanation'], dtype='object')

In [5]:
df_train = pd.read_csv('/kaggle/input/dataset/Dataset/train_df.tsv',sep='\t')
df_train.columns = ["image",'captions','sarcasm_explanations']
df_train

,image,captions,sarcasm_explanations
0,880425829246922752,'proud to have a <user> who attacks private ci...,it's sad to have a <user> who attacks private ...
1,690915881082343424,'the view from my house ........ yay # pasnow...,the author has a terrible view of everything c...
2,915228456757059585,'these # colombo # jaffna # vavuniya luxury bu...,these # colombo # jaffna # vavuniya luxury bus...
3,494194068998468686_25639236,"Thanks North Carolina, I was worried we weren'...",the author was worried that they were gonna hi...
4,847004394612776965,'junk mail : free viagra . stripper wants to m...,the author is annoyed with such junk mail.
...,...,...,...
2977,806820636517691392,'well done # us # syria',US shouldn't have attacked hospital in mosul a...
2978,824024594755518464,oh yes fios let 's upgrade our wi-fi for even...,the author isn't interested to upgrade their w...
2979,720315804189732864,'inspiring confidence during the registration ...,The registration website doesn’t seem very sec...
2980,992845208666844469_180900586,Don't I feel #safe in our #government offices....,it's unsafe to have such wires laying around.


In [6]:
train_image_list = df_train['image'].to_list()

df_explanations = df_explanations[df_explanations['image'].isin(train_image_list)]

In [7]:
df_train = pd.merge(df_explanations,df_train,how='inner',on='image')
df_train.shape

(2982, 4)

In [8]:
df_test = pd.read_csv('/kaggle/input/dataset/Dataset/test_df.tsv',sep='\t')
df_test.columns = ["image",'captions','sarcasm_explanations']
df_test

,image,captions,sarcasm_explanations
0,700183392969756672,'oh i so love working late from home # work #...,the author hates working late from home.
1,928753954745475072,'yeaah ! buddy o miracle worker # infj emoji_...,"your anxiety is not cured when someone says ""d..."
2,935133439011049473,'rt <user> : something different ..... a delay...,the author is pissed to watch a full train lea...
3,933466049697198080,'oh really linkedin ? thanks for the super use...,the author doesn't find such notifications fro...
4,699788403940978689,'our beautiful view along pch ! # pch # highw...,the author can't really see the view because o...
...,...,...,...
346,1011850445043480900_1580447253,âï¸âï¸âï¸âï¸âï¸âï¸âï¸âï...,it isn't a cool week if it's 100 degrees.
347,636237012294327831_256939246,She's nothing like her dad... ;) #lovethem #fa...,"she's exactly like her dad, both are making si..."
348,899685897251069952,'this eclipse is even cooler than i thought it...,the author is disappointed with this eclipse s...
349,878368201221914624,'<user> app radar is definitely right on targe...,<user> app radar isn't right on target.


In [9]:
df_explanations = pd.read_csv('/kaggle/input/dataset/Dataset/explanations.csv')

def remove_jpg_extension(image_name):
    final_image_name = image_name.split('.')[0]
    return final_image_name

df_explanations['image'] = df_explanations['image'].apply(remove_jpg_extension)

df_explanations.drop('Unnamed: 2',axis=1,inplace=True)

test_image_list = df_test['image'].to_list()

df_explanations = df_explanations[df_explanations['image'].isin(test_image_list)]

In [10]:
df_test = pd.merge(df_explanations,df_test,how='inner',on='image')
df_test = df_test[234:]
df_test.shape

(117, 4)

In [11]:
df_train.iloc[0]['llm_explanation']

'**Tweet Description**\n\nThe image displays a tweet with a GIF of Kanye West looking serious.\n\n* The tweet reads, "They behaved like two passionate footballers who want to win games."\n* Attached to the tweet is a GIF of Kanye West looking serious.\n* Below the original tweet, there is another tweet.\n* The second tweet features a GIF of Kanye West with a serious expression and reads, "Nice to see Ashley Young & Dele Alli behaving like a couple of kids in a playground #MANTOT."'

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
# import torch
# import transformers
# from datetime import datetime
# from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
# from datasets import load_dataset
from peft import prepare_model_for_kbit_training,LoraConfig,PeftModel,get_peft_model

In [14]:
model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                          device_map="auto",
                                          quantization_config=bnb_config,
                                          trust_remote_code=True)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [16]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

trainable params: 262364160 || all params: 1521392640 || trainable%: 17.244999949519933


In [17]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=8, #Rank
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, config)

In [18]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result


def generate_and_tokenize_prompt(data_point):
    full_prompt = f"""
    We are given common sense explanation of a sarcastic image and caption. Provide me with sarcasm explanation in the image.
    
    ### Image Explanation:
    {data_point["llm_explanation"]}
    
    ### Caption:
    {data_point["captions"]}
    
    ### Sarcasm Explanation:
    {data_point["sarcasm_explanations"]}
    """
    return tokenize(full_prompt)

def generate_and_tokenize_prompt_eval(data_point):
    full_prompt = f"""
    We are given common sense explanation of a sarcastic image and caption. Provide me with sarcasm explanation in the image.
    
    ### Image Explanation:
    {data_point["llm_explanation"]}
    
    ### Caption:
    {data_point["captions"]}
    
    ### Sarcasm Explanation:
    """
    return tokenize(full_prompt)


In [19]:
from tqdm import tqdm

tokenized_test_dataset = []

for i in tqdm(range(df_test.shape[0])):

    data_entry = {
        "llm_explanation" : df_test.iloc[i]['llm_explanation'],
        "captions" : df_test.iloc[i]['captions'],
        "sarcasm_explanations" : df_test.iloc[i]["sarcasm_explanations"]
        }
    
    # data_entry = {
    #     "premise": eval_dataset["premise"][i],
    #     "hypothesis": eval_dataset["hypothesis"][i],
    #     "label": eval_dataset["label"][i]
    # }
    
    tokenized_test_dataset.append(generate_and_tokenize_prompt_eval(data_entry))
    
    

100%|██████████| 117/117 [00:00<00:00, 306.72it/s]


In [20]:
from peft import PeftModel, LoraConfig
import torch
from tqdm import tqdm

ft_model = PeftModel.from_pretrained(model, "/kaggle/input/phi2_project/transformers/default/1", config = LoraConfig())
ft_model.eval()

model_output = []

def process_input(v):
    """Helper function to process input values"""
    if isinstance(v, list):
        return torch.tensor(v).unsqueeze(0).to('cuda')
    elif isinstance(v, torch.Tensor):
        return v.to('cuda')
    else:
        return v

def prepare_model_input(model_input):
    """Extract and process only the required model inputs"""
    valid_keys = ['input_ids', 'attention_mask']
    filtered_input = {
        k: process_input(v) 
        for k, v in model_input.items() 
        if k in valid_keys
    }
    return filtered_input

for model_input in tqdm(tokenized_test_dataset):
    processed_input = prepare_model_input(model_input)
    
    with torch.no_grad():
        generated_output = ft_model.generate(
            **processed_input,
            max_new_tokens=512,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            num_beams=5
        )[0]
        
        decoded_text = tokenizer.decode(generated_output, skip_special_tokens=True)
        model_output.append(decoded_text)

print("\nFirst few generations:")
for i, output in enumerate(model_output[:3]):
    print(f"\nGeneration {i+1}:")
    print(output)

100%|██████████| 117/117 [2:40:40<00:00, 82.40s/it] 


First few generations:

Generation 1:

      We are given common sense explanation of a sarcastic image and caption. Provide me with sarcasm explanation in the image.
      
      ### Image Explanation:
      The image shows a notification that the user has been recommended as an influencer to follow on LinkedIn. The notification is from LinkedIn, and it includes a photo of Bill Gates, who is the co-chair of the Bill & Melinda Gates Foundation.
      
      ### Caption:
      'oh really linkedin? thanks for the super useful notification!    # incaseyoucouldnttell # stopabusingnotifications # influencer # ugh emoji_177'
      
      ### Sarcasm Explanation:
      The user sarcastically replies to the LinkedIn notification that they have been recommended as an influencer to follow on LinkedIn. The user sarcastically thanks LinkedIn for the "super useful" notification. The user also sarcastically mentions that they are not interested in being an influencer and that they are tired of rece

In [41]:
model_output[29]

'\n      We are given common sense explanation of a sarcastic image and caption. Provide me with sarcasm explanation in the image.\n      \n      ### Image Explanation:\n      This infographic presents a forecast for Phoenix, Arizona, titled "Another cool week in Arizona," featuring a predominantly white background with black text and yellow circles, accompanied by gray lines and clouds.\n\nThe forecasted temperatures range from 82°F to 115°F, with the highest temperatures occurring on Wed, Thu, and Fri. The image conveys an air of irony with the label "cool week," given the unusually high temperatures.\n      \n      ### Caption:\n      â\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8fâ\x98\x80ï¸\x8f ð\x9f\x98\x93ð\x9f\x98®ð\x9f\x98²ð\x9f\x98µð\x9f\x92¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f\x94¥ð\x9f

In [21]:
import json

with open('/kaggle/working/model_output.json', 'w') as f:
    json.dump(model_output, f)

from IPython.display import FileLink
FileLink(r'model_output.json')


/kaggle/working/model_output.json

In [22]:
dt = pd.DataFrame(model_output)
dt.to_csv('model_output_old_117_234')

In [64]:
l = []
for i in range(len(model_output)):
    try:
        print(i)
        sarcasm_explanation = model_output[i].split("### Sarcasm Explanation:")[1].split('###')[0]
    except (IndexError, KeyError):
        sarcasm_explanation = ' '  
    l.append(sarcasm_explanation)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [37]:
dt = pd.DataFrame(l)
dt.to_csv('l_234_315')

In [ ]:
def read_file_to_string(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()  
        return content
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

file_path = "/kaggle/input/d-117-234/model_output_old_117_234_0"  

file_content = read_file_to_string(file_path)

print(file_content)


In [40]:
def count_occurrences(file_path, substring):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()  
        return content.count(substring) 
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0

file_path = "/kaggle/input/d-117-234/model_output_old_117_234_0"  

substring = "### Sarcasm Explanation:"

count = count_occurrences(file_path, substring)

print(f"The substring '{substring}' appears {count} times in the file.")


The substring '### Sarcasm Explanation:' appears 116 times in the file.


In [45]:
def count_occurrences(file_path, substring):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()  
        return content.count(substring)  
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0

file_path = "/kaggle/input/d-117-234/model_output_old_117_234_0"  

substring = "### Caption:"

count = count_occurrences(file_path, substring)

print(f"The substring '{substring}' appears {count} times in the file.")


The substring '### Caption:' appears 117 times in the file.


An error occurred: '<' not supported between instances of 'int' and 'NoneType'
The number of '### Image Explanation:' before the specific text is: 0


In [65]:
def extract_sections_between_image_explanations(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        sections = content.split("### Image Explanation:")
        
        extracted_sections = []
        
        for section in sections[1:]:
            full_section = "### Image Explanation:" + section.strip()
            
            next_split_index = full_section.find("### Image Explanation:", len("### Image Explanation:"))
            if next_split_index != -1:
                full_section = full_section[:next_split_index]
            
            extracted_sections.append(full_section.strip())
        
        return extracted_sections
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

file_path = "/kaggle/input/o-0-117/model_output_old_0_117"  

sections0 = extract_sections_between_image_explanations(file_path)

for idx, section in enumerate(sections, 1):
    print(f"Section {idx}:\n{section}\n")


Section 1:
### Image Explanation:The image depicts a large outdoor screen broadcasting a football game or movie, with a crowd of people watching on a field. The field is covered in blankets and pillows, with some people lying down to watch the game.

In the background, a large scoreboard and a brick building with a sign that reads ""town pump"" are visible, set against a pink and orange cloudy sky. The overall atmosphere suggests a fun and relaxed outdoor event, possibly a movie night or a casual gathering to watch a sporting event.
      
      ### Caption:
     'seriously why would you ever live here...    # lastbestplace # gocatsgo'
      
      ### Sarcasm Explanation:
      The caption is a sarcastic comment on why someone would choose to live in the town depicted in the image. The use of the hashtag #lastbestplace suggests that the town is not actually a desirable place to live, despite the relaxed and fun atmosphere depicted in the image. The phrase ""seriously why would you eve

In [66]:
def extract_sections_between_image_explanations(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        sections = content.split("### Image Explanation:")
        
        extracted_sections = []
        
        for section in sections[1:]:  
            full_section = "### Image Explanation:" + section.strip()
            
            next_split_index = full_section.find("### Image Explanation:", len("### Image Explanation:"))
            if next_split_index != -1:
                full_section = full_section[:next_split_index]
            
            extracted_sections.append(full_section.strip())
        
        return extracted_sections
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

file_path = "/kaggle/input/d-117-234/model_output_old_117_234_0"  
sections1 = extract_sections_between_image_explanations(file_path)

for idx, section in enumerate(sections, 1):
    print(f"Section {idx}:\n{section}\n")


Section 1:
### Image Explanation:The image depicts a large outdoor screen broadcasting a football game or movie, with a crowd of people watching on a field. The field is covered in blankets and pillows, with some people lying down to watch the game.

In the background, a large scoreboard and a brick building with a sign that reads ""town pump"" are visible, set against a pink and orange cloudy sky. The overall atmosphere suggests a fun and relaxed outdoor event, possibly a movie night or a casual gathering to watch a sporting event.
      
      ### Caption:
     'seriously why would you ever live here...    # lastbestplace # gocatsgo'
      
      ### Sarcasm Explanation:
      The caption is a sarcastic comment on why someone would choose to live in the town depicted in the image. The use of the hashtag #lastbestplace suggests that the town is not actually a desirable place to live, despite the relaxed and fun atmosphere depicted in the image. The phrase ""seriously why would you eve

In [67]:
l1 = []
for i in range(len(sections1)):
    try:
        print(i)
        sarcasm_explanation = sections1[i].split("### Sarcasm Explanation:")[1].split('###')[0]
    except (IndexError, KeyError):
        sarcasm_explanation = ' '  
    l1.append(sarcasm_explanation)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [68]:
l0 = []
for i in range(len(sections0)):
    try:
        print(i)
        sarcasm_explanation = sections0[i].split("### Sarcasm Explanation:")[1].split('###')[0]
    except (IndexError, KeyError):
        sarcasm_explanation = ' ' 
    l0.append(sarcasm_explanation)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [69]:
l3 = l0+l1+l

In [70]:
len(l3)

351

In [71]:
import json


with open('/kaggle/working/final_output.json', 'w') as f:
    json.dump(l3, f)

from IPython.display import FileLink
FileLink(r'final_output.json')

/kaggle/working/final_output.json

In [72]:
import json

def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)  
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

file_path = "/kaggle/working/final_output.json"  
json_data = read_json_file(file_path)

if json_data:
    for idx, item in enumerate(json_data, 1):
        print(f"Item {idx}:\n{item}\n")
else:
    print("No valid JSON data found.")


Item 1:

      The caption is a sarcastic comment on why someone would choose to live in the town depicted in the image. The use of the hashtag #lastbestplace suggests that the town is not actually a desirable place to live, despite the relaxed and fun atmosphere depicted in the image. The phrase ""seriously why would you ever live here"" is a sarcastic way of expressing disbelief or incredulity at the idea of anyone choosing to live in the town. The use of the hashtag #gocatsgo is a reference to the popular TV show ""Game of Thrones"", and suggests that the town is not actually a desirable place to live, despite the relaxed and fun atmosphere depicted in the image. Overall, the caption is a sarcastic comment on the idea of anyone choosing to live in the town depicted in the image.
      
      

Item 2:

      The caption is a sarcastic response to the politician's plea for donations. The use of ""awww poor cruz is broke"" and ""cry a river of tears"" implies that the politician is ac